In [1]:
import os
import cv2

In [2]:
# Initialize OpenCV object
positive_video_name = 'positive.avi'
negative_video_name = 'negative.avi'
videos = [negative_video_name, positive_video_name]

# Define image transforms
import torch
import numpy as np
from torchvision import transforms as T
import pandas as pd
from pandas import DataFrame 

transform0 = T.ToPILImage()
transform1 = T.ToTensor()        

means = []
stds = []


n = 0
data = {'name': [], 'target': []}
for i, v in enumerate(videos):
    # Read videos
    cap = cv2.VideoCapture(os.path.join('../data/videos/', v))
    frame_width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    frame_height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(frame_width, frame_height, frame_count, end=",")
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        if ret:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # np.array: (h240, w320, C3) 
            frame = transform1(frame) # to tensor (c3, h240, w320)
#             means.append(np.array([frame[0,:,:].mean().item(), frame[1,:,:].mean().item(), frame[2,:,:].mean().item()]))
#             stds.append(np.array([frame[0,:,:].std().item(), frame[1,:,:].std().item(), frame[2,:,:].std().item()]))
            
            frame = transform0(frame) # to PIL
            frame.save('../data/handnet/imgs/{0:04d}.png'.format(n))
            data['name'].append('{0:04d}.png'.format(n))
            data['target'].append(0 if i == 0 else 1)
            
            n += 1
            
        else:
            break
        
        cv2.destroyAllWindows()
        
        
# mean = np.array(means).mean(axis=0)
# std = np.array(stds).mean(axis=0)

# print('mean: ', mean)
# print('std: ', std)

data_df = DataFrame(data)
data_df.to_csv('../data/handnet/data_df.csv', sep=',')
    

320.0 240.0 2808,320.0 240.0 3648,

In [ ]:
# negative: 2808 frames
# mean:  [0.63031351 0.61356543 0.61106094]
# std:  [0.2586194  0.265984   0.26036792]

# positive: 3647
# mean:  [0.64892411 0.62235533 0.61335696]
# std:  [0.25001224 0.25962306 0.25678512]

In [ ]:
# merge mean & std of positive and negiative
n_mean=  np.array([0.63031351, 0.61356543, 0.61106094])
n_std=  np.array([0.2586194,  0.265984,  0.26036792])
n_mean = n_mean * 2808
n_std = n_std * 2808

p_mean=  np.array([0.64892411, 0.62235533, 0.61335696])
p_std= np.array([0.25001224, 0.25962306, 0.25678512])
p_mean = p_mean * 3647
p_std =  p_std * 3647

mean = (n_mean + p_mean)/(2808+3647)
std = (n_std + p_std)/(2808+3647)

In [ ]:
print(mean, std)

In [ ]:
# [0.64082828 0.61853162 0.61235816] [0.25375645 0.26239014 0.25834368]